<font face="Calibri" size="2"> <i>Open SAR Toolkit - Tutorial 5, version 1.0, September 2019. Andreas Vollrath, ESA/ESRIN phi-lab</i>
</font>

![title](auxiliary/header_image.PNG)

--------

# OST Tutorial V
## How to do a batch processing of Sentinel-1 SLC data based on bursts

--------

**Short description**

This notebook introduces you to the general workflow of OST for the batch processing of SLC data using the *Sentunel1_SCLBatch* class. This is a subclass of the *Sentinel1* class, and thus inherits all the functionality of the *Generic* and the *Sentinel1* classes. In addition, calibrated backscatter will be jointly produced together with the interferometric coherence and the dual-pol H/A/alpha decomposition.

--------

**Requirements**

- a PC/Mac with at least 16GB of RAM
- about 50GB of free disk space
- a Copernicus Open Data Hub user account, valid for at least 7 days (https://scihub.copernicus.eu)
--------

**NOTE:** all cells that have an * after its number can be executed without changing any code. 

### 1* - Import of the class

In [ ]:
# this is the s1Project class, that basically handles all the workflow from beginning to the end
from ost import Sentinel1_SLCBatch

### 2 - Define your project parameters and intialize the class


As a subclass of the *Sentinel-1* class, the *Sentinel1_SLCBatch* class inherit all attributes nd methods from the *Generic* and *Sentinel1* class. 
In addition it provides attributes and methods for the ARD definition, the inventory of bursts, as well as the processing of the bursts to ARD, and the subsequent creation of timeseries, timescans and large-scale mosaics. 

In [ ]:
# get home folder
import os
from os.path import join
home = os.getenv('HOME')
# create a processing directory
project_dir = join('{}'.format(home), 'OpenSarToolkit', 'Tutorial_5')

# define a custom temp directory and use an eodata storage on CREODIAS (only applies if you have both set up)
#temp_dir = '/ram'
#dias_data='/eodata'

# apply function with buffer in meters
from ost.helpers import vector
lat, lon = '50.68', '10.92'
aoi = vector.latlon_to_wkt(lat, lon, buffer_meter=6000, envelope=True)
#----------------------------
# Time of interest
#----------------------------
# we set only the start date to today - 30 days
start = '2018-01-01'
end = '2018-01-20'

# create s1Project class instance
s1_batch = Sentinel1_SLCBatch(
    project_dir=project_dir, 
    aoi=aoi, 
    start=start, 
    end=end, 
    product_type='SLC', 
    #temp_dir=temp_dir,
    #data_mount=dias_data, 
    ard_type='OST Plus')

### 3* - Data inventory

We take the known functionality from the *Sentinel1* class to search for available acquisitions. 

In [ ]:
#---------------------------------------------------
# for plotting purposes we use this iPython magic
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (19, 19)
#---------------------------------------------------

# search command
s1_batch.search()
# we plot the full Inventory on a map
s1_batch.plot_inventory(transperancy=.1)

In [ ]:
s1_batch.refine()

### 4* - Select a mosaic key

We select one of our mosaic key, seen in the summarising information of the previous refine command, and check the coverage by plotting the footprints on a map.

In [ ]:
pylab.rcParams['figure.figsize'] = (13, 13)


key = 'DESCENDING_VVVH'
s1_batch.refined_inventory_dict[key]
s1_batch.plot_inventory(s1_batch.refined_inventory_dict[key], 0.3)

### (5*) - Download of scenes

The download routine, already known from the *Sentinel1* class, is used to download the scenes.

In [ ]:
s1_batch.download(s1_batch.refined_inventory_dict[key])

### 6 - Creating a burst inventory

Sentinel-1 does not come with predefined footprints. This is a problem when working on larger areas and coherence should be processed, which results from two images of the same orbit taken at different times. However, there is a smaller part of the imagery referred to as bursts, which is uniquelly defined on its position by the AnxTime attribute in the Sentinel-1 annotation files. By creating a database, that holds a burst id, consisting of orbit, subswath and AnxTime, and the repsective scene and its burst number, we can subsequently process burst by burst. 

The advantages are:
- less memory needs
- can be more easily scaled for parallel processing on clusters
- processing is reduced by considering only the bursts actually overlapping with the AOI
- by using SLC data, polarimetric decompositions as well as interferometric coherence can be generated

In [ ]:
s1_batch.create_burst_inventory(key=key, refine=True)

In [ ]:
pylab.rcParams['figure.figsize'] = (13, 13)
s1_batch.plot_inventory(s1_batch.burst_inventory, transperancy=0.1)

In [ ]:
print('Our burst inventory holds {} bursts to process.'.format(len(s1_batch.burst_inventory)))
print('------------------------------------------')
print(s1_batch.burst_inventory.head())

### 7 - Analysis Ready-Data format for various SLC products and the multi-temporal data products

ARD formats for SLC data, apart from the backscatter are not well defined. Nonetheless, OST allows to produce the dual-pol H/A/Alpha decomposition as well as the interferometric coherence of consecutive acquisitions in an automated manner. 

In addition, the time-series of the resulting products can be produced, first burst-by-burst, and then followed by mosaicking. The same is true for the generation of timescan, where 5 different statistics can be computed.

For the creation of time-series 2 additional parameters are added:
- *to_db_mt*: output the time-series data in dB scale
- *mt_speckle-filter*: apply the multi-temporal speckle filter (standard Snap parameters, i.e. Refined Lee)

For the creation of the timescans 2 additional parameters are added:
- *metrics*: The stastical measures applied over each timeseries. By default all measures, consisting of average (avg), maximum (max), minimum (min), standard deviation (std) and Coeffcient of Variance (cov)
- *outlier_removal*: In some cases images are affected by artifacts with strong signals. For the calculation of the timescan layers, extreme values can be excluded setting this parameter to True.


For this example exercise we customize some of the ARD parameters to create aminimal processing example and save on prcessing time. 

In [ ]:
print('---------------------------------')
print(' Original OST Plus ARD parameters')
print(s1_batch.ard_parameters)
print('---------------------------------')

s1_batch.ard_parameters['product_type'] = 'GTCgamma'
s1_batch.ard_parameters['ls_mask_create'] = False
s1_batch.ard_parameters['ls_mask_apply'] = False
s1_batch.ard_parameters['to_db'] = False
s1_batch.ard_parameters['speckle_filter'] = False
s1_batch.ard_parameters['pol_speckle_filter'] = False
s1_batch.ard_parameters['mt_speckle_filter'] = False
s1_batch.ard_parameters['to_db_mt'] = True
s1_batch.ard_parameters['resolution'] = 30

print('---------------------------------')
print(' Customized ARD parameters')
print(s1_batch.ard_parameters)
print('---------------------------------')

### 8* - Launch processing

The *burst_to_ard* method allows to execute the full workflow consiting of:
1 Burst to ARD: each burst undergoes the ARD generation routine.
2 ARD to Timeseries: a timeseries is created for all overlapping bursts
3 Timeseries to Timescan: the multi-temporal metrics are calculated for all burst timeseries
4 Mossaicking: All bursts of the timeseries as well as the timescans are mosaicked to a single file

**Note** The overwrite parameter allows to start from scratch, meaning that all previously processed data is deleted. Otherwise the routine will restart from where it stopped. At the moment this aplies only for the Burst to ARD routine, which is the most processing intense.

In [ ]:
s1_batch.burst_to_ard(timeseries=True, timescan=True, mosaic=True, overwrite=False)